In [19]:
# import re
# import numpy as np
# import pandas as pd
# from pprint import pprint
# from nltk.corpus import stopwords

# # Gensim
# import gensim
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel

# # spacy for lemmatization
# import spacy

# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline

import sys
import spacy
import pickle
import numpy as np
from nltk.corpus import stopwords
from gensim.corpora import Dictionary
from sklearn.linear_model import LinearRegression
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.wrappers import LdaMallet
import logging
import os


In [20]:
# import nltk
# nltk.download()

In [21]:
os.listdir('/mallet-2.0.8/mallet-2.0.8/')

['bin',
 'build.xml',
 'class',
 'dist',
 'lib',
 'LICENSE',
 'Makefile',
 'pom.xml',
 'README.md',
 'sample-data',
 'src',
 'stoplists',
 'test']

In [27]:
class Serialization:
    @staticmethod
    def save_obj(obj, name):
        """
        serialization of an object
        :param obj: object to serialize
        :param name: file name to store the object
        """
        with open('Topic Model Data/' + name + '.pkl', 'wb') as fout:
            pickle.dump(obj, fout, pickle.HIGHEST_PROTOCOL)
        # end with
    # end def

    @staticmethod
    def load_obj(name):
        """
        de-serialization of an object
        :param name: file name to load the object from
        """
        with open(name + '.pkl', 'rb') as fin:
            return pickle.load(fin)
    
    @staticmethod
    def get_file_data(dirname, file_name):
        with open(f'{dirname}{file_name}', 'r', encoding="UTF-8") as file:
            data = file.readlines()
            return [line.strip() for line in data if len(line) > 1] 

        # end with
    # end def

# end class

In [53]:
class DataUtils:

    def __init__(self):
        self.ALLOWED_POSTAGS = ['NOUN', 'ADJ', 'VERB', 'ADV']
        self.MAX_WORD_RANK = 10000  # based on Wikipedia word frequencies
        self.MIN_WORD_RANK = 300  # based on Wikipedia word frequencies
        self.MIN_POST_LENGTH = 50  # after cleanup

        self.nlp = spacy.load('en_core_web_lg', disable=['tokenizer', 'parser', 'ner'])
        self.word_ranks = Serialization.load_obj('dict.word.ranks')
        self.stop_words = stopwords.words('english')

    # end def

    def lemmatization(self, text):
        """https://spacy.io/api/annotation"""
        output = list()
        for token in self.nlp(text):
            if token.pos_ not in self.ALLOWED_POSTAGS: continue
            output.append(token.lemma_)
        # end for
        return output
    # end def

    def load_and_clean_data(self, dirname, filename):
        clean_data = list()
        data = Serialization.get_file_data(dirname, filename)
        print(len(data))
        for line in data:
#             print('processing data for week', week)
#             for i, post in enumerate(data[week]):
            clean = self.extract_post_clean_data(line)
            if len(clean) < self.MIN_POST_LENGTH: 
                continue
            clean_data.append(clean)
            # end for
        # end for

        print('extracted', len(clean_data), 'posts')
        return clean_data

    # end def

    def extract_post_clean_data(self, post):
        clean_post = list()
        for word in self.lemmatization(post):
            # exclude stopwords and words with atypical length
            if word in self.stop_words or len(word) not in range(4, 15): continue
            # exclude too common or too rare words
            if self.word_ranks.get(word, 0) < self.MIN_WORD_RANK or \
                self.word_ranks.get(word, sys.maxsize) > self.MAX_WORD_RANK: continue

            clean_post.append(word)
        # end for
        return clean_post

    # end def

# end class


In [44]:

class TopicModeling():

    MALLET_PATH = '/mallet-2.0.8/mallet-2.0.8/bin/mallet'

    @staticmethod
    def find_best_number_of_topics(data):
        dictionary = Dictionary(data)
        corpus = [dictionary.doc2bow(text) for text in data]

        scores = dict()
        for topics in range(2, 10, 1):
            print('performing topic modeling with', topics, 'topics')
            ldamodel = LdaMallet(TopicModeling.MALLET_PATH, corpus=corpus, num_topics=topics, id2word=dictionary)
            coherence_model = CoherenceModel(model=ldamodel, texts=data, coherence='c_v')
            coherence = coherence_model.get_coherence()
            scores[topics] = coherence
        # end for

        print('coherence scores: the higher, the better:', scores)
    # end def

    @staticmethod
    def generate_topics(data, topics, gender):
        dictionary = Dictionary(data)
        corpus = [dictionary.doc2bow(text) for text in data]
        print('performing topic modeling with', topics, 'topics')
        ldamodel = LdaMallet(TopicModeling.MALLET_PATH, corpus=corpus, num_topics=topics, id2word=dictionary)
        ldamodel.save('ldamodel.' + gender + '.' + str(topics))

    # end def

# end class


def explore_temporal_trends(topics, gender):
    model = LdaMallet.load('ldamodel.' + gender + '.' + str(topics))
    topic_words = model.show_topics(num_topics=topics, num_words=100, formatted=False)
    words_only = [(tp[0], [wd[0] for wd in tp[1]]) for tp in topic_words]

    words = list()
    for tp in words_only: words.extend(tp[1])

    weekly_stats = dict()
    data = Serialization.load_obj('week2comments.' + gender)
    for week in range(len(data)):
        if week not in data: continue
        print('processing data for week', week)
        week_data = ' '.join(data[week]).lower()
        total = len(week_data.split())

        for word in set(words):
            current_word_stats = weekly_stats.get(word, list())
            current_word_stats.append(float(week_data.split().count(word))/total)
            weekly_stats[word] = current_word_stats
        # end for
    # end for

    for word in weekly_stats:
        y = np.array(weekly_stats[word])
        x = np.array([i for i in range(len(weekly_stats[word]))]).reshape(-1, 1)
        model = LinearRegression().fit(x, y)
        print(word, model.coef_[0])
    # end for

# end def




In [45]:
def sort_files(dirname, cutoff=1):
    sort_map = {}
    max_int = cutoff
    for file in os.listdir(dirname):
        dot_index = file.index(".")
        week = int(file[4:dot_index])
        if week >= cutoff:
            if sort_map.get(week) is None:
                sort_map[week] = [file]
            else:
                sort_map[week].append(file)
        if week >= max_int:
            max_int = week
    sorted_files = []
    for i in range(cutoff, max_int+1):
        sorted_files.extend(sort_map[i])
    return sorted_files
        

In [54]:
# todo: a reasonably good tutorial on topic modeling is here -
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

# todo: the code assumes mallet topic modeling project in <current_dir>/mallet-2.0.8/
# todo: can download from http://mallet.cs.umass.edu/topics.php


utils = DataUtils()
data_files = sort_files('Region3/', 11)
#     print(data_files)


In [55]:
for week_file in data_files:
    print(week_file)
# the code assumes week2comments pkl file (e.g., week2comments.M.pkl) in pickle/
# the file is a dictionary pickle object: week -> list of post and comments for that week
    data = utils.load_and_clean_data('Region3/', f'{week_file}')
    Serialization.save_obj(data, 'clean.data.' + week_file)
    


Week11.Midwest.txt
911
extracted 43 posts
Week11.Northeast.txt
2028
extracted 109 posts
Week11.Pacific.txt
4467
extracted 256 posts
Week11.Rocky Mountain.txt
284
extracted 13 posts
Week11.Southeast.txt
1408
extracted 74 posts
Week11.Southwest.txt
1135
extracted 57 posts
Week12.Midwest.txt
3219
extracted 184 posts
Week12.Northeast.txt
4265
extracted 256 posts
Week12.Pacific.txt
5507
extracted 399 posts
Week12.Rocky Mountain.txt
1061
extracted 50 posts
Week12.Southeast.txt
3487
extracted 202 posts
Week12.Southwest.txt
2110
extracted 112 posts
Week13.Midwest.txt
10657
extracted 744 posts
Week13.Northeast.txt
9782
extracted 695 posts
Week13.Pacific.txt
10863
extracted 803 posts
Week13.Rocky Mountain.txt
2847
extracted 217 posts
Week13.Southeast.txt
8885
extracted 662 posts
Week13.Southwest.txt
5704
extracted 392 posts
Week14.Midwest.txt
9938
extracted 811 posts
Week14.Northeast.txt
10112
extracted 779 posts
Week14.Pacific.txt
11392
extracted 869 posts
Week14.Rocky Mountain.txt
2467
extract

In [ ]:
def get_regions(dirname):
    all_region_files = os.listdir(dirname)
    one_week = [file for file in all_region_files if file.startswith('Week12')]
    period = one_week[0].index('.')
    regions = [file[period+1:] for file in one_week]
    regions.sort()
    return regions

def get_weeks_per_region(region):
    all_pkl = os.listdir('Topic Model Data/')
    region_pkl = [file for file in all_pkl if file.endswith(f'{region}.txt.pkl')]
    region_pkl.sort()
    return region_pkl
    
    

In [ ]:
modeling = TopicModeling()
regions = get_regions('Region3/')
for region in regions:
    region_pkls = get_weeks_per_region(region)
    for file in region_pkls:
        data = Serialization.load_obj(file[:-4])
        modeling.find_best_number_of_topics(data)
    break

In [ ]:
modeling = TopicModeling()
data = Serialization.load_obj('clean.data.' + region)
modeling.find_best_number_of_topics(data)  # todo: inspect coherence scores and decide on topics number

topics = 8  # based on manual inspection in the previous step
modeling.generate_topics(data, topics, gender)

explore_temporal_trends(topics, gender)

In [34]:
with open(f'Region3/Week11.Midwest.txt', 'r', encoding="UTF-8") as file:
    data = file.readlines()
    x = [line.strip() for line in data if len(line) > 1]
    for y in x:
        print(y)
        print("NEW")

Didn't the Potus just ask the military how the progress on a corona vaccine is coming along like yesterday? Or is that not asking the government to fix our problems? I won't ask why the military is involved rather than a coalition of pharma representation experts all working together.
NEW
Fear is healthy but the way the msm paints this on some outlets makes it sound like the end of the world. The key word is "transparency", and with 90% of the media outlets in this country being owned by 6 mega corps there isn't much "transparency" to be had. (And sorry if I sound a bit pissy. I just got thru watching Waffle Warren lie her ass off on Ellen about "what we need to do about the Corona virus".) Edit, adding, Blueskaisunshine you might like this small sub, REDDIT And here's a link to Corona stats the dashboard from John Hopkins University LINK
NEW
That's the problem. This article doesn't give any of that data. It's just a piece asking random people from across the state if they think they c

...really? Here's all the statistics, I would think that you would have seen these as you've referenced death rate stats. LINK If no young or no people without pre-existing conditions had died the numbers for those demographics would be 0% You responded to me saying that it's not true that it ONLY affects old and sick people. It doesnt.
NEW
lol. theres arent more reported cases because there is basically zero mass/rapid testing happening in the US and the rollout has been completely botched. you cant diagnose what you cant detect. rest assured, there is community transmission happening in the States and we're not getting a full picture of the issue because of incompetence. all the while the virus is being allowed to spread by people who dont know what it is they have. once we get rapid testing you can expect these numbers to skyrocket, and by then it will be nearly impossible to contain because we basically squandered an entire month.
NEW
This loony started a cult, and a lot of people 

I know that's true to an extent, but a government isn't going to shut down schools and tank the economy solely because a few people are freaking out. There's obviously a threat there that they recognize, and they want to get ahead of it. I get it, but most people I know are living their day to day lives still, even as the threat of COVID looms. I do agree that some people overreact, but I've seen far more people saying "This is pretty much exactly like the flu, everyone overreacting is falling for media BS", which is frustrating because it's certainly not the flu, and I haven't really seen much mainstream media coverage that is misleading or outright false. Unfortunately, I don't have the expertise to speak about it myself and have to rely on the explanations of those who know more than me, but just yesterday, the [WHO updated their worldwide mortality rate to 3.4%]LINK and their initial estimate was between .7-4% depending on the quality of care that a patient receives. This is likely

But if what we're saying in this thread and has been hypothesized in the news - that the Covid-19 virus is much further spread than we think, with many people going along with mild symptoms, and only the most extreme cases being reported - than the mortality rate would actually be much lower than currently reported. Also the WHO has said that the mortality rate so far has a broad range from about 0.7% - 4% depending on the health care system. They also believe it is far less transmissible than the common flu.
NEW
This information is plastered all over the NYT site. Just search for their Coronavirus live update blog. I'd link but I'm on mobile and have a paywall. The defining characteristic is that this is unlike a cold or the regular flu, it's a severe upper respiratory infection with lesions on the inside of your lungs, a dry cough, and shortness of breath.
NEW
Speaking as someone in Korea I think it's because they are testing an insane amount of people. They are forcing tests on the 

I'm starting to get a little worried. Even though I'm young (26) and might not get as sick or die from COVID-19--I don't know how I will go get tested or treated if I get infected because I no longer have health insurance... I rarely catch the seasonal flu and I think I have a strong immune system but if I do get it I could probably get a lot of people around me sick because I cannot afford to go to the doc. nor stop going to work. Sorry in advance!
NEW
The costs for his hospital visit are ridiculous, he went because of his concern for Covid-19. Are you arguing because he couldn't tell if his symptoms were flu or not he should pay $1400? Of course you are attacking the source of my info, I suppose you're superior critical thinking just lets you google until you find the source that agrees with you.
NEW
Yes and he also didn't get the covid-19 test, which is free. He insisted on the hospital giving him a normal flu test. Going to the ER for a flu test is idiotic, he could've gone to an u

...

I had never heard that the virus leaves behind permanent damage. I have heard that a number of Chinese people who had recovered got infected again, but not that there's permanent damage. Can anyone familiar with that claim let me know more, please?
NEW
To be fair you weren't wrong. 3 cases were from last week that the results just came in as they had corona. Now they're linked to the death. And I believe 7 came from the senior home that got infected. 1 a man in his 50s with underlying condition. And another one no info released.
NEW
If I'm remembering right, on the CDC website, they look for 3 things to determine if a patient has recovered. 1. No fever, and not currently using any fever reducing medicine. 2. No persistent cough. 3. Consecutive negative respiratory test results for presence of the virus.
NEW
Not to be all THE SKY IS FALLING, but what do they mean "recovered"? I assume that means the virus is no longer detectable in their blood...but what of (if any) lung damage was/is t

It's because he's gonna cancel shows due to the Coronavirus very soonly. It's what u call a cash grab
NEW
reading about historical components and reactions, reading opinions from doctors from Johns Hopkins and Harvard, etc. I'm not a doctor - never claimed to be. I am well read though. Nothing I've said above is panic-inducing nor sensationalist. The last "big" Coronavirus incidents were SARS and MERS. Sars had a Fatality rate of 10%, MERS 34%. To give comparison, Avian flu was 65%. Spanish flu (1918) was estimated to be 20% though there were times when it was much higher. The "regular" flu is, I believe .01% and Covid is based on current info at about 2%, though some are saying it's closer to 1%. Even so, that's 10x - 20x higher a fatality than the Flu. *BUT* that's not high enough, so it can linger, people can get moderate cases of it. Now their are theories that i read this morning that are saying that it's been in the US for up to 6 weeks without manifesting - that means it's been 

It might be Coronavirus related but I have seen some good travel deals and I have a flexible work schedule. Someone tell me about a good one they have seen lately. Warm is best for me. But I would be open to flying anywhere. Except Detroit. Stop trying to make that happen Spirit Airlines I am not flying there! Let me know what you have found.
NEW
Open thread for your opinions. I'm only concerned because Mexico apparently doesn't have any reported. If it hits them hard many will wish we had a border wall. Oh the irony of the name too.
NEW
UMKC Chancellor addresses coronavirus in email Feb. 27, 2020 To Our University Community: As news about the coronavirus, or COVID-19, continues to emerge from across the globe, we wanted to share with you some important information to help you stay safe and support others. It's important to note that both Kansas and Missouri continue to have no confirmed coronavirus cases. Officials from the Centers for Disease Control and Prevention (CDC), however, ha

UMKC Chancellor addresses coronavirus in email Feb. 27, 2020 To Our University Community: As news about the coronavirus, or COVID-19, continues to emerge from across the globe, we wanted to share with you some important information to help you stay safe and support others. It's important to note that both Kansas and Missouri continue to have no confirmed coronavirus cases. Officials from the Centers for Disease Control and Prevention (CDC), however, have said that clusters of infection are likely to arise in the U.S. The virus originated in China and is spreading across the globe. Travel to both China and South Korea is now restricted by the U.S. government, and no new university-sponsored travel to China or South Korea will be approved until further notice. In addition, clusters have arisen in Italy and Iran. UMKC is continuing to monitor CDC travel advisories. UMKC is working closely with our partners in public health to track the spread of the virus and preparing to respond swiftly 

There's a confirmed case of corona here and the big wigs are trying to cover it up. Sucks I don't play that game. It's not as bad as people think, but it's here.
NEW
Hello REDDIT There has been a recent uptick in posts related to the Novel Coronavirus (COVID-19). The mod team is going to take the same approach to these as we are with political postings, discussion should be specifically related to the MSA only. No "what if" or "does anyone" type of posts, no posts about selling or hoarding of masks, sanitizers, or other health-related items. REDDIT would be your place for discussion for items not directly related to the Cincinnati area Please note: The CDC has neither confirmed nor denied that a goetta and cheese omelet from Price Hill Chili is an effective deterrent to the Coronavirus, but why take a chance?
NEW
Is there anyone in here who has insights to this? Many economists and firms are predicting a recession if this virus gets semi-bad. How could this impact Cincinnati and the jo